# Imports and definitions

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import shutil
import sys
import cv2
from tqdm import tqdm



# K-fold validation
from sklearn.model_selection import KFold

# Classification acuracy metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# Keras imports for models
from keras import models
from keras import layers

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D

from tensorflow.keras.optimizers import Adam, RMSprop, Adamax, SGD

from keras import applications
from keras.applications import imagenet_utils

from keras.utils import to_categorical

# Sklearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import VotingClassifier

# Sklearn regressors
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.utils import shuffle
from sklearn.ensemble import StackingRegressor

#Sklearn PCA
from sklearn.decomposition import PCA

In order to run the notebook I've included a pickle file for the dataframe used to train the models (Dev-set) and for the Test-set

In [ ]:
os.getcwd()

'C:\\Users\\rugby\\OneDrive\\Documents\\DCU- Artificial Intelligence\\Machine Learning\\Assignment\\Code'

 Here we are defining our different directories where the data is stored so that we may easily switch between **them**

In [ ]:
base_dir = "C:/Users/rugby/Datasets/CA684/Dev-set"
train_dir = os.path.join(base_dir, "Train-Set")
test_dir = os.path.join(base_dir, "Test-Set")
os.chdir(train_dir)

In [ ]:
os.getcwd()

'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set'

Finding where the features are located relevant to our current directory and adding the path to each to a dictionary object where the keys are the feature name for easy access

In [ ]:
feature_folders = {}
for feature_folder in os.listdir(os.getcwd()):
    feature_folders[str(feature_folder)] = os.path.join(os.getcwd(),str(feature_folder))
  

In [ ]:
feature_folders

{'Aesthetic Features': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\Aesthetic Features',
 'C3D': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\C3D',
 'Captions': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\Captions',
 'ColorHistogram': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\ColorHistogram',
 'ColorHistogramImages': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\ColorHistogramImages',
 'frames': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\frames',
 'Ground-truth': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\Ground-truth',
 'HMP': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\HMP',
 'HMPImages': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\HMPImages',
 'HMPImages - Copy': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\HMPImages - Copy',
 'HMP_Images_Val': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-set\\Train-Set\\HMP_Images_Val',
 'HOG': 'C:\\Users\\rugby\\Datasets\\CA684\\Dev-se

# **Loading Ground Tuths:**
Loading in the ground truths first as they are among the most important

In [ ]:
groundTruths_file = os.path.join(train_dir, "Ground-truth/ground-truth.csv")
groundTruthsdf = pd.read_csv(groundTruths_file)
groundTruthsdf


,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10
...,...,...,...,...,...
5995,video7488.webm,0.921,33,1.000,9
5996,video7489.webm,0.909,53,0.839,31
5997,video7491.webm,0.713,33,0.818,11
5998,video7492.webm,0.954,34,1.000,16


Test set ground truths template was updated to match the format of the Dev set ground truths, this was done manually in excel

In [ ]:
groundTruthsTest_file = os.path.join(test_dir, "Ground-truth/ground_truth_template.csv")
groundTruthsTestdf = pd.read_csv(groundTruthsTest_file)
groundTruthsTestdf

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video7494.webm,NaN,33,NaN,12
1,video7495.webm,NaN,34,NaN,10
2,video7496.webm,NaN,32,NaN,13
3,video7497.webm,NaN,33,NaN,10
4,video7498.webm,NaN,33,NaN,10
...,...,...,...,...,...
1995,video10004.webm,NaN,34,NaN,17
1996,video10005.webm,NaN,34,NaN,9
1997,video10006.webm,NaN,34,NaN,12
1998,video10007.webm,NaN,34,NaN,12


#Feature Extraction

## Moving videos
Using the dev set ground truths file, videos are copied into the relevant directories. i.e. Any vidoe in the dev set is moved to the videos folder in the train directory and any videos in the test set is moved to the videos folder in the test directory

In [ ]:
for video in groundTruthsdf['video']:
    shutil.move(os.path.join("C:/Users/rugby/Datasets/CA684/Videos/Complete_Sources", video), os.path.join(feature_folders['Videos'], video))

test_videos = os.path.join(test_dir, 'Videos')

for video in groundTruthsTestdf['video']:
    shutil.move(os.path.join("C:/Users/rugby/Datasets/CA684/Videos/Complete_Sources", video), os.path.join(test_videos, video))

## Extracting frames

In [ ]:
def extract_frames(videos, sources_path, frames_path):

    for video_name in videos:
        
        # Path to each video, video directory and filename
        video_path = os.path.join(sources_path, video_name)
        # Removing .wbm from video files so that the actual video name can be used to save each frame
        video_name_short = video_name.split('.webm')[0]

        print('Video: {}'.format(video_path))

        # Read Video!
        success = True
        video = cv2.VideoCapture(video_path)
        success, image = video.read()

        # Frame
        frame_id = int(video.get(1))
        cv2.imwrite("{}/{}-frame-{}.jpg".format(frames_path, video_name_short, frame_id), image)
        
        # Frames per second
        fps = int(video.get(cv2.CAP_PROP_FPS))
        
        # Collect 1 frame every 1 second, 8 frames in total over the 8 seconds
        seconds = 1
        multiplier = fps * seconds

        # While we have a video i.e. One is loaded, iterate over the video and
        # save an image everytime a second has passed in the clip
        while success:
            # Frame
            frame_id = int(video.get(1))
            if frame_id % multiplier == 0:
                # Save each frame as an image with the video name and associated frame as the filename
                cv2.imwrite("{}/{}-frame-{}.jpg".format(frames_path, video_name_short, frame_id), image)

            # Read Video!
            success, image = video.read()

        video.release()


# Dev set
dev_videos = feature_folders['Videos']
dev_videosdf = groundTruthsdf['video']
dev_frames_path = os.path.join(train_dir, 'frames')
store_frames(dev_videosdf, dev_videos, dev_frames_path)

# Test set
test_videos = os.path.join(test_dir, 'Videos')
test_videosdf = groundTruthsTestdf['video']
test_frames_path = os.path.join(test_dir, 'frames')
store_frames(test_videos, test_videos, test_frames_path)

This is just to clear any GPU sessions currently running on my laptop before I begin extracting the features

In [ ]:
print(tf.config.list_physical_devices('GPU')) 
print(tf.test.gpu_device_name())

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available() # True/False

# Or only check for gpu's with cuda support
tf.test.is_gpu_available(cuda_only=True) 

## Extracting ResNet features

In [ ]:
# Using a pre-trained ResNet model without its top level classifier as we are not looking for a classification in this instance
model = applications.ResNet50(include_top=False, weights='imagenet', pooling='avg')

videos = list(groundTruthsdf['video'])

# Each video has frames extracted and the names of these frames is the same except for the number at the end
# This takes all images of frames from the same videos and groups them together in order
image_paths = [
            os.path.join(dev_frames_path, video.split('.webm')[0] + '-frame-{}.jpg'.format(frame))
            for video in videos
            for frame in [1, 24, 48, 72, 96, 120, 144, 168]
        ]

# Changing to the ResNet features folder in our train directory
os.chdir(feature_folders['ResNet'])

frame = 0
Video_images = []
for image in tqdm(image_paths):
    videoName = str(image.split('\\')[3].split('-')[0])
    frame = frame +1
    
    # Loading each image in and resizeing it to (244, 244) as this is the input shape ResNet expects
    # Adapted from https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1
    input_image = tf.keras.preprocessing.image.load_img(image, target_size=(224, 224))
    input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    input_image = imagenet_utils.preprocess_input(input_image)
    
    Video_images.append(input_image)
    
    # Since a frame was extracted for every second there are 8 frames per video. 
    # predictions are made in batches of 8 so that they are done on a video by video basis.
    # Once all 8 images have been appended to our training batch this batch is then passed to our ResNet model
    if frame == 8:
        Video_images = np.vstack(Video_images)
        features = model.predict(Video_images, batch_size=8)
        # Reshapes our outout into arrays of with 2048 values as this is the output from the ResNet model
        # Resnet Features will be in the shape (8, 2048), this represents 2048 features for each of the 8 frames extracted 
        features = features.reshape((features.shape[0], 2048))
        
        # Files are then dumped into a pickle file and saved for storage
        with open(videoName, 'wb') as handle:
            pickle.dump(features, handle)
        Video_images = []
        frame = 0
        
    
    
    
    

100%|██████████| 48000/48000 [27:32<00:00, 29.06it/s] 


In [ ]:
model = applications.ResNet50(include_top=False, weights='imagenet', pooling='avg')

videos = list(groundTruthsTestdf['video'])

image_paths = [
            os.path.join(test_frames_path, video.split('.webm')[0] + '-frame-{}.jpg'.format(frame))
            for video in videos
            for frame in [1, 24, 48, 72, 96, 120, 144, 168]
        ]

os.chdir(os.path.join(test_dir, 'ResNet'))

frame = 0
Video_images = []
for image in tqdm(image_paths):
    videoName = str(image.split('\\')[3].split('-')[0])
    frame = frame +1
    
    # Loading each image in and resizeing it to (244, 244) as this is the input shape ResNet expects
    # Adapted from https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1
    input_image = tf.keras.preprocessing.image.load_img(image, target_size=(224, 224))
    input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    input_image = imagenet_utils.preprocess_input(input_image)
    
    Video_images.append(input_image)
    
    # Since a frame was extracted for every second there are 8 frames per video. 
    # predictions are made in batches of 8 so that they are done on a video by video basis.
    # Once all 8 images have been appended to our training batch this batch is then passed to our ResNet model
    if frame == 8:
        Video_images = np.vstack(Video_images)
        features = model.predict(Video_images, batch_size=8)
        # Reshapes our outout into arrays of with 2048 values as this is the output from the ResNet model
        # Resnet Features will be in the shape (8, 2048), this represents 2048 features for each of the 8 frames extracted 
        features = features.reshape((features.shape[0], 2048))
        
        # Files are then dumped into a pickle file and saved for storage
        with open(videoName, 'wb') as handle:
            pickle.dump(features, handle)
        Video_images = []
        frame = 0
        
    
    
    
    

100%|██████████| 16000/16000 [10:01<00:00, 26.62it/s]


# **Loading in features**
Defining the functions given to use to read inn some of the video features

In [ ]:
# define all the functions in this section 

def vname2ID(vnames):
    """Parse video digital id from its name
    vnames: a list contains file names"""
    vid = [ os.path.splitext(vn)[0][5:] for vn in vnames]
    return vid

def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D

def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

def read_ColorHistogram(fname):
    """Scan Color Histogram from file
    Input file contains RGB histogram,
    Return a matrix of (3,256)"""
    RGB_Hist = np.zeros((3,256))
    with open(fname) as f:
        i_l = 0 # line index
        for line in f:
            pairs = line.split()
            hist_dict = {int(p.split(':')[0]):float(p.split(':')[1]) for p in pairs}
            for idx in hist_dict.keys():
                RGB_Hist[i_l,idx] = hist_dict[idx]
            i_l += 1
    return RGB_Hist

def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df

def read_aesthetic(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            aesthetic =[float(item) for item in line.split(',')] # convert to float type, using default separator
    return aesthetic

def read_LBP(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            LBP =[float(item) for item in line.split()] # convert to float type, using default separator
    return LBP

def read_HOG(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            HoG =[float(item) for item in line.split()] # convert to float type, using default separator
    return HoG

def read_RESNET(fname):
    with open(fname, 'rb') as handle:
        RESNET = pickle.load(handle)
        RESNET = RESNET.flatten() # Loads precomputed array and flattens it from an (8, 2048) array to a (16348) dimensionaly 1D array
    return RESNET
        

## Feature dictionaries
Next we will go through our features and load each of them into their own dictionary object using the previously defined functions above

In [ ]:
os.chdir(feature_folders['C3D'])

C3D_Dict = {"video":[],"C3D":[]}

for file in sorted(os.listdir(os.getcwd())):
    C3D_Dict["video"].append(os.path.splitext(file)[0])
    C3D_Dict["C3D"].append(np.array(read_C3D(file)))

In [ ]:
aesthetic_mean = os.path.join(feature_folders['Aesthetic Features'], 'dev-set_aesthetic-features', 'aesthetic_feat_dev-set_mean', )
aesthetic_median = os.path.join(feature_folders['Aesthetic Features'], 'dev-set_aesthetic-features', 'aesthetic_feat_dev-set_median', )

os.chdir(aesthetic_mean)

aesthetic_mean_Dict = {"video":[],"aesthetic_mean":[]}

for file in os.listdir(os.getcwd()):
    aesthetic_mean_Dict["video"].append(os.path.splitext(file)[0])
    aesthetic_mean_Dict["aesthetic_mean"].append(np.array(read_aesthetic(file)))
    
os.chdir(aesthetic_median)

aesthetic_median_Dict = {"video":[],"aesthetic_median":[]}

for file in os.listdir(os.getcwd()):
    aesthetic_median_Dict["video"].append(os.path.splitext(file)[0])
    aesthetic_median_Dict["aesthetic_median"].append(np.array(read_aesthetic(file)))

In [ ]:
os.chdir(feature_folders['ResNet'])

ResNet_Dict = {"video":[],"ResNet":[]}

for file in sorted(os.listdir(os.getcwd())):
    ResNet_Dict["video"].append(os.path.splitext(file)[0])
    ResNet_Dict["ResNet"].append(read_RESNET(file))

In [ ]:
# These dictionaries are then added to a list and we will iteratively combine these dictionaries with our ground truths dataframe
datasets = [C3D_Dict, aesthetic_mean_Dict, aesthetic_median_Dict, ResNet_Dict]

In [ ]:
groundTruths_file = os.path.join(train_dir, "Ground-truth/ground-truth.csv")

In [ ]:
groundTruthsdf = pd.read_csv(groundTruths_file)
groundTruthsdf['video'] = groundTruthsdf['video'].str.rstrip('.webm')
groundTruthsdf

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3,0.924,34,0.846,13
1,video4,0.923,33,0.667,12
2,video6,0.863,33,0.700,10
3,video8,0.922,33,0.818,11
4,video10,0.950,34,0.900,10
...,...,...,...,...,...
5995,video7488,0.921,33,1.000,9
5996,video7489,0.909,53,0.839,31
5997,video7491,0.713,33,0.818,11
5998,video7492,0.954,34,1.000,16


## Classification Labels
Here we create new labels for our classification task

In [ ]:
# These values were chosen in order to preserve a data balance, it would be possible to just take the most significant digit from each score and use this as our label i.e. 0.924 becomes 9
# but this would cause us to have a large data imbalance as the majority of videos have a high short-term memorability score

def ClassifyLT(x):
    if  x <=.799:
        return 0
    elif x >.799:
        return 1

    
def ClassifyST(x):
    if x <=.87:
        return 0
    elif x >.87:
        return 1
    



groundTruthsdf['ST_Round'] = groundTruthsdf['short-term_memorability'].apply(lambda x: ClassifyST(x))
groundTruthsdf['LT_Round'] = groundTruthsdf['long-term_memorability'].apply(lambda x: ClassifyLT(x))

groundTruthsdf

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,ST_Round,LT_Round
0,video3,0.924,34,0.846,13,1,1
1,video4,0.923,33,0.667,12,1,0
2,video6,0.863,33,0.700,10,0,0
3,video8,0.922,33,0.818,11,1,1
4,video10,0.950,34,0.900,10,1,1
...,...,...,...,...,...,...,...
5995,video7488,0.921,33,1.000,9,1,1
5996,video7489,0.909,53,0.839,31,1,1
5997,video7491,0.713,33,0.818,11,0,1
5998,video7492,0.954,34,1.000,16,1,1


Once all of our features have been loaded they are combined with our ground truths dataframe

In [ ]:
Dataset = groundTruthsdf

for data in datasets:
    temp = pd.DataFrame(data)
    temp
    Dataset = Dataset.merge(temp, left_on='video', right_on='video')

In [ ]:
Dataset

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,ST_Round,LT_Round,C3D,aesthetic_mean,aesthetic_median,ResNet
0,video3,0.924,34,0.846,13,1,1,"[0.02024942, 0.0015778, 0.00082625, 0.00094509...","[0.18389, 0.713733, 0.202302, 0.610132, 0.1242...","[0.180874, 0.715626, 0.19893, 0.610453, 0.1216...","[0.27144504, 0.45918608, 0.035905946, 4.542650..."
1,video4,0.923,33,0.667,12,1,0,"[0.000118, 0.00089075, 0.00018769, 4.543e-05, ...","[0.147255, 0.550718, 0.460299, 0.424981, 0.341...","[0.146076, 0.55007, 0.460729, 0.424397, 0.3418...","[0.9976212, 0.3619391, 0.2518191, 0.114101954,..."
2,video6,0.863,33,0.700,10,0,0,"[0.01176522, 0.00074577, 0.00078353, 1.328e-05...","[0.063933, 0.826933, 0.214129, 0.719341, 0.130...","[0.063898, 0.827439, 0.211925, 0.720297, 0.129...","[0.7898796, 0.09988401, 0.020203931, 4.1379895..."
3,video8,0.922,33,0.818,11,1,1,"[0.00022343, 0.00016499, 7.35e-06, 1.615e-05, ...","[0.367527, 0.133869, 0.746074, 0.169317, 0.696...","[0.380539, 0.134683, 0.746891, 0.171604, 0.697...","[0.0059119402, 0.27636862, 0.095939755, 0.1347..."
4,video10,0.950,34,0.900,10,1,1,"[9.006e-05, 0.00061494, 0.00343634, 0.00128092...","[0.402878, 0.389358, 0.564687, 0.400378, 0.471...","[0.427258, 0.381448, 0.582746, 0.409475, 0.489...","[0.9489327, 1.3563356, 0.0, 0.33917212, 0.1328..."
...,...,...,...,...,...,...,...,...,...,...,...
5995,video7488,0.921,33,1.000,9,1,1,"[0.35724622, 0.07598178, 0.00302532, 2.772e-05...","[0.4086, 0.194645, 0.713681, 0.283166, 0.65350...","[0.413464, 0.195451, 0.71372, 0.282848, 0.6534...","[0.5562924, 0.60748005, 0.0, 2.5067296, 0.0560..."
5996,video7489,0.909,53,0.839,31,1,1,"[0.19280288, 0.00093812, 0.00506054, 1.35e-06,...","[0.265468, 0.188006, 0.451723, 0.160899, 0.415...","[0.264377, 0.192016, 0.448979, 0.164919, 0.412...","[0.5345125, 0.45983094, 0.012904617, 0.3444094..."
5997,video7491,0.713,33,0.818,11,0,1,"[2.589e-05, 0.00012192, 2.781e-05, 2.608e-05, ...","[0.178087, 0.220956, 0.671676, 0.1793, 0.60967...","[0.177078, 0.221348, 0.67387, 0.180848, 0.6131...","[0.8621345, 0.29973385, 0.039232306, 3.5872233..."
5998,video7492,0.954,34,1.000,16,1,1,"[0.26509121, 0.09653918, 5.971e-05, 0.00022735...","[0.113677, 0.585402, 0.496425, 0.475082, 0.352...","[0.113838, 0.585274, 0.496719, 0.474999, 0.352...","[0.531669, 0.3867722, 0.0, 4.139823, 0.2704557..."


## Saving feature set
Once combined the dataframe is dumped to a pickle file and saved for later use

In [ ]:
os.chdir('C:\\Users\\rugby\\OneDrive\\Documents\\DCU- Artificial Intelligence\\Machine Learning\\Assignment\\Code\\Pickle Files')
os.getcwd()

'C:\\Users\\rugby\\OneDrive\\Documents\\DCU- Artificial Intelligence\\Machine Learning\\Assignment\\Code\\Pickle Files'

In [ ]:
filename = "FullFeatureSet.pickle"
with open(filename, 'wb') as handle:
    pickle.dump(Dataset, handle)




## Loading our feature set

In [ ]:
os.chdir('C:\\Users\\rugby\\OneDrive\\Documents\\DCU- Artificial Intelligence\\Machine Learning\\Assignment\\Code\\Pickle Files')
os.getcwd()

with open('FullFeatureSet.pickle', 'rb') as handle:
        temp = pickle.load(handle)
        Dataset = temp

In [ ]:
Dataset

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,ST_Round,LT_Round,C3D,aesthetic_mean,aesthetic_median,ResNet
0,video3,0.924,34,0.846,13,1,1,"[0.02024942, 0.0015778, 0.00082625, 0.00094509...","[0.18389, 0.713733, 0.202302, 0.610132, 0.1242...","[0.180874, 0.715626, 0.19893, 0.610453, 0.1216...","[0.27144504, 0.45918608, 0.035905946, 4.542650..."
1,video4,0.923,33,0.667,12,1,0,"[0.000118, 0.00089075, 0.00018769, 4.543e-05, ...","[0.147255, 0.550718, 0.460299, 0.424981, 0.341...","[0.146076, 0.55007, 0.460729, 0.424397, 0.3418...","[0.9976212, 0.3619391, 0.2518191, 0.114101954,..."
2,video6,0.863,33,0.700,10,0,0,"[0.01176522, 0.00074577, 0.00078353, 1.328e-05...","[0.063933, 0.826933, 0.214129, 0.719341, 0.130...","[0.063898, 0.827439, 0.211925, 0.720297, 0.129...","[0.7898796, 0.09988401, 0.020203931, 4.1379895..."
3,video8,0.922,33,0.818,11,1,1,"[0.00022343, 0.00016499, 7.35e-06, 1.615e-05, ...","[0.367527, 0.133869, 0.746074, 0.169317, 0.696...","[0.380539, 0.134683, 0.746891, 0.171604, 0.697...","[0.0059119402, 0.27636862, 0.095939755, 0.1347..."
4,video10,0.950,34,0.900,10,1,1,"[9.006e-05, 0.00061494, 0.00343634, 0.00128092...","[0.402878, 0.389358, 0.564687, 0.400378, 0.471...","[0.427258, 0.381448, 0.582746, 0.409475, 0.489...","[0.9489327, 1.3563356, 0.0, 0.33917212, 0.1328..."
...,...,...,...,...,...,...,...,...,...,...,...
5995,video7488,0.921,33,1.000,9,1,1,"[0.35724622, 0.07598178, 0.00302532, 2.772e-05...","[0.4086, 0.194645, 0.713681, 0.283166, 0.65350...","[0.413464, 0.195451, 0.71372, 0.282848, 0.6534...","[0.5562924, 0.60748005, 0.0, 2.5067296, 0.0560..."
5996,video7489,0.909,53,0.839,31,1,1,"[0.19280288, 0.00093812, 0.00506054, 1.35e-06,...","[0.265468, 0.188006, 0.451723, 0.160899, 0.415...","[0.264377, 0.192016, 0.448979, 0.164919, 0.412...","[0.5345125, 0.45983094, 0.012904617, 0.3444094..."
5997,video7491,0.713,33,0.818,11,0,1,"[2.589e-05, 0.00012192, 2.781e-05, 2.608e-05, ...","[0.178087, 0.220956, 0.671676, 0.1793, 0.60967...","[0.177078, 0.221348, 0.67387, 0.180848, 0.6131...","[0.8621345, 0.29973385, 0.039232306, 3.5872233..."
5998,video7492,0.954,34,1.000,16,1,1,"[0.26509121, 0.09653918, 5.971e-05, 0.00022735...","[0.113677, 0.585402, 0.496425, 0.475082, 0.352...","[0.113838, 0.585274, 0.496719, 0.474999, 0.352...","[0.531669, 0.3867722, 0.0, 4.139823, 0.2704557..."


# Feature Processing
Before our features are passed to any models for training they are first converted to a numpy array and the values inside are normalized.

In [ ]:
def normalize(data):
    normalize = data / np.linalg.norm(data)
    return normalize

# Depending on whether a single feature is to be used or if a combination of features are being used, the following functions handle combinging features by appending them together
# If a value actually a label and not a feature it is not normalized

def process_single_attribute_Values(Xin, value, label='attr'):
    newX = []

    if label == 'attr':
        for index in range(len(Xin)):
            newX.append((Xin[index][value]))
    else:
        for index in range(len(Xin)):
            newX.append(Xin[index][value])

    newX = np.array(newX)
    
    return newX

# If multiple different features are to be combined they are passed to this function which, depending on how many are to be joined will iterate through all of the feeatures
# and append them all to the same numpy array

def process_join_attribute_Values(Xin, num):
    newX = []

    if num ==2:
        for index in range(len(Xin)):
            newX.append(np.concatenate((np.array(normalize(Xin[index][0])), np.array(normalize(Xin[index][1])))))
    
    elif num ==3:
        for index in range(len(Xin)):
            newX.append(np.concatenate((np.array((Xin[index][0])), np.array((Xin[index][1])), np.array((Xin[index][2])))))
            
    elif num ==4:
        for index in range(len(Xin)):
            newX.append(np.concatenate((np.array((Xin[index][0])), np.array((Xin[index][1])), np.array((Xin[index][2])), np.array((Xin[index][3])))))

    newX = np.array(newX)
    
    return newX


In [ ]:
# Here we are defining the function we use to create our datasets for k-fold cross validation where we are either passing a single feature
# or we are passing multiple to be combined

def K_fold_processing(X, Y, num):
    
     # 1
    if num ==1:
        X_Attr = process_single_attribute_Values(X, 0)

        Y_ST = (process_single_attribute_Values(Y,0, 'label'))

        Y_LT = (process_single_attribute_Values(Y,1, 'label'))
        
    else:
        X_Attr = process_join_attribute_Values(X, num)

        Y_ST = (process_single_attribute_Values(Y,0, 'label'))

        Y_LT = (process_single_attribute_Values(Y,1, 'label'))

    return X_Attr, Y_ST, Y_LT


In [ ]:
# Here we are just defining a function to apply PCA in order to reduce our feature set where the 2 parameters are how much information we want to keep from our original feature
# and the feature to be reduced
def apply_PCA(n, X):
    pca = PCA(n)
    return pca.fit_transform(X)


Using our Dataset dataframe from earlier we are using the resNet extracted features as our feature and the rounded ST and LT classification labels as our target variables

In [ ]:
Y = Dataset[['ST_Round','LT_Round']].values # targets

X = Dataset[['ResNet']].values

# K-Fold cross validation

X_Attr1, Y_ST1, Y_LT1 = K_fold_processing(X, Y, 1)

# As the ResNet features are very big we apply PCA to reduce them down to a more manageable number for training
# before we preform any PCA ResNet features are 16,384 long.
# After PCA we are left with an array of size (1958), much shorter and will greatly improve training times for our models
pca = PCA(n_components=2000)
X_Attr1 = pca.fit_transform(X_Attr1)

Data = {
    'ResNet':{
        'X':X_Attr1,
        'Y_ST':Y_ST1,
        'Y_LT':Y_LT1
    }
}


# Classification Model Selection
Several different models were tested and evaluated before the final decision was made for which model to choose. Both Keras and Sklearn models were evaluated

## Keras models

In [ ]:
# A shallow network was found to give relatively good accuracy scores  and train in a relatively quick timeframe
# Finetuning the number of nodes and dropout rate only provides minimal imporvements if any.
# Also adding additional layers did not do much in the way of improving accuracy so it was decided to keep the model as simple as possible for testing

def get_model():
    model = Sequential()
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(32,activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(4, activation="relu"))

    model.add(Dense(2, activation="softmax"))
    
    return model


In [ ]:
# This function converts softmax probabilities to a single value representing the predicted label.
# E.g. [0.3, 0.7] becomes [1] as index 1 is the larger value
# This is so the predictions can be used to caluclate several different sklearn metrics in order to accuractley compare and contrast the different models

# Takes a list of predictions in the fornm of softmax probabilities and conversts each entry to a single 1 D array, it does this for every prediction passed in

def convertSoftmaxToIndexes(preds):
    output = list()
    
    for row in preds: 
        tempIndex=0
        tempVal=row[0]
        index=0
        for val in row:
            if val >tempVal:
                tempVal=val
                tempIndex=index
            index=index+1
        output.append(tempIndex)
        
    return output

In [ ]:
kf = KFold(n_splits=6)
kf.get_n_splits(X_Attr1, Y_ST1)

Fold = 1  
Acc_Scores = []
Recall_Scores = []
Precision_Scores = []
F1_Scores = []

for train_index, test_index in kf.split(X_Attr1):
    print("Fold: ", Fold)
    X_train, X_test = X_Attr1[train_index], X_Attr1[test_index]
    y_train, y_test = Y_ST1[train_index], Y_ST1[test_index]
    
    # one hot encode outputs
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    model = get_model()
    
    # The Adam optimizer was found to achieve the best results, generally as a rule of thumb I've found that the newest optimizers (Most recently published) tend
    # to preform the best compared to some of the older, more simple optimizers. This is not always the case though
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4), metrics=['acc'])
    model.fit(
        x=X_train, y=y_train,
        epochs=30,
        verbose=0)
    
    score = model.evaluate(X_test, y_test, verbose = 2) 
    preds = model.predict(X_test)
    preds = convertSoftmaxToIndexes(preds)
    y_test = convertSoftmaxToIndexes(y_test)

    # Each metic is calculated and added to an array so that the scores for each fold are saved and can be averaged
    Acc_Scores.append(score[1])
    Recall_Scores.append(recall_score(y_test, preds))
    Precision_Scores.append(precision_score(y_test, preds))
    F1_Scores.append(f1_score(y_test, preds))
    Fold = Fold + 1
    
print("Average Accuracy Score: ", np.mean(Acc_Scores))
print("Average Recall Score: ", np.mean(Recall_Scores))
print("Average Precision Score: ", np.mean(Precision_Scores))
print("Average F1 Score: ", np.mean(F1_Scores))

    

Fold:  1
32/32 - 0s - loss: 0.6617 - acc: 0.6660
Fold:  2
32/32 - 0s - loss: 0.6537 - acc: 0.6540
Fold:  3
32/32 - 0s - loss: 0.6349 - acc: 0.6670
Fold:  4
32/32 - 0s - loss: 0.6349 - acc: 0.6660
Fold:  5
32/32 - 0s - loss: 0.6670 - acc: 0.6440
Fold:  6
32/32 - 0s - loss: 0.6397 - acc: 0.6680
Average Accuracy Score:  0.660833328962326
Average Recall Score:  0.6639950963456894
Average Precision Score:  0.6413718745782198
Average F1 Score:  0.6522971894367866


Altering the provided Get_Score() function so that it returns the Spearman score and does not print the score for each run

In [ ]:
def Altered_Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            return score_mat.iloc[1][0]
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

In [ ]:
# The paramenters for the SVC model were found by using a grid search 
parameters ={
    'SVC': {
        'C': [0.1, 1.0, 100.0, 10000.0],
        'kernel': ['linear', 'rbf', 'sigmoid'],
        'gamma': ['scale', 'auto'], 
    }
}

temp = GridSearchCV(
                    SVC(),
                    parameters['SVC'],
                    cv=3,
                    n_jobs=-1,
                    verbose=1
                )
temp = temp.fit(X_Attr1, Y_ST1)
print("Best parameters Short-Term: {}".format(temp.best_params_))

models ={
    'AdaB': AdaBoostClassifier(random_state=1),
    'RFR': RandomForestClassifier(class_weight='balanced', random_state=1),
    'GBR': GradientBoostingClassifier(),
    'Bag': BalancedBaggingClassifier(),
    'ET': ExtraTreesClassifier(class_weight='balanced', random_state=1),
    'SVC': SVC(gamma='scale', C=1.0, kernel='rbf')   
}


## Sklearn models
Next a range of sklearn classifiers were tested to see how each preformed on the classification task

In [ ]:
# Declaring a dictionary that stores the best scores for each metric as well as the model and data used to achieve this score
# However in this case ResNet was found to achieve the best results for this classification problem
Best = {
    'Model_Acc':'','Data_Acc':'', 'Acc':0, 
    'Model_Recall':'','Data_Recall':'', 'Recall':0, 
    'Model_Precision':'','Data_Precision':'', 'Precision':0, 
    'Model_F1':'','Data_F1':'', 'F1':0
}

# Iterates through all of the datasets in the Data dictionary. More can be added to this dictionary in order to evaluate training on different features
for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    # Iterates through all of the models in the model dictionary, more can be evaluated by adding them to this dictionary 
    for model in models:
        print("_______________ Model: ", model, "_______________")

        temp = models[model]
            
        Fold = 1  
        
        # Each list is initialized for each model
        SP_Scores = []
        Acc_Scores = []
        Recall_Scores = []
        Precision_Scores = []
        F1_Scores = []
        
        # Using K-fold cross validation to split the data into 6 different folds
        for train_index, test_index in kf.split(X):
            print("Fold: ", Fold)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            
            # Traing and predicting 
            temp = temp.fit(X_train, y_train)
            preds = temp.predict(X_test)
            
            # Calculating the different classification metrics
            SP_Scores.append(Altered_Get_score(preds, y_test))
            Acc_Scores.append(temp.score(X_test, y_test))
            Recall_Scores.append(recall_score(y_test, preds))
            Precision_Scores.append(precision_score(y_test, preds))
            F1_Scores.append(f1_score(y_test, preds))
            Fold = Fold +1
            
        # Once a moel has been trained on all 6 folds the average scores for each metric is calculated in printed to screen
        # to give a good estimation on how the model may preform on previously unseen data
        print("Average Spearmans Score: ", np.mean(SP_Scores))
        
        print("Average Accuracy Score: ", np.mean(Acc_Scores))
        if np.mean(Acc_Scores) > Best['Acc']:
            Best['Acc'] = np.mean(Acc_Scores)
            Best['Model_Acc'] = model
            Best['Data_Acc'] = data
            
        print("Average Recall Score: ", np.mean(Recall_Scores))
        if np.mean(Recall_Scores) > Best['Recall']:
            Best['Recall'] = np.mean(Recall_Scores)
            Best['Model_Recall'] = model
            Best['Data_Recall'] = data
            
        print("Average Precision Score: ", np.mean(Precision_Scores))
        if np.mean(Precision_Scores) > Best['Precision']:
            Best['Precision'] = np.mean(Precision_Scores)
            Best['Model_Precision'] = model
            Best['Data_Precision'] = data
            
        print("Average F1 Score: ", np.mean(F1_Scores))
        if np.mean(F1_Scores) > Best['F1']:
            Best['F1'] = np.mean(F1_Scores)
            Best['Model_F1'] = model
            Best['Data_F1'] = data
        print("")

# Finally the best scores and models are printed to screen
print("_____Best Scores and models_____")
for key in Best:
    print(key, ':', Best[key])


_____  ResNet _____
_______________ Model:  AdaB _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.24254137040526405
Average Accuracy Score:  0.6218333333333333
Average Recall Score:  0.6150770937626386
Average Precision Score:  0.6038237349001134
Average F1 Score:  0.609332026878028

_______________ Model:  RFR _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.14522837809302952
Average Accuracy Score:  0.575
Average Recall Score:  0.4287147607162079
Average Precision Score:  0.5770135139528942
Average F1 Score:  0.49131223741619356

_______________ Model:  GBR _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.291636330594385
Average Accuracy Score:  0.6451666666666668
Average Recall Score:  0.6627780930841781
Average Precision Score:  0.622167176858584
Average F1 Score:  0.6416230036632381

_______________ Model:  Bag _______________
Fold:  1


## Ensemble approach
An ensemble approach was then used to try and increase the accuracy scores attained but did not end up yielding significant improvements. Several different combinations were used based off of the scores previously attained when 
evaluating each model individually

In [ ]:
# A similar approach to above is used except the model in this case is an ensemble of the previous models

model1 = BalancedBaggingClassifier(base_estimator=SVC())
model2 = RandomForestClassifier(class_weight='balanced', random_state=1)
model3 = GradientBoostingClassifier(random_state=1)
model4 = ExtraTreesClassifier(class_weight='balanced', random_state=1)
model5 = SVC(gamma='scale', C=1.0, kernel='rbf')   
    

for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)
    
    Fold = 1  
    SP_Scores = []
    Acc_Scores = []
    Recall_Scores = []
    Precision_Scores = []
    F1_Scores = []
    for train_index, test_index in kf.split(X):
        print("Fold: ", Fold)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        Ensemble1 = VotingClassifier(estimators=[('Bag', model1), ('RF', model2), ('GBC', model3), ('ETC', model4), ('SVC', model5)], voting='hard', weights=[1,1,2,1,3])
        Ensemble1 = Ensemble1.fit(X_train, y_train)
        preds = Ensemble1.predict(X_test)
        SP_Scores.append(Altered_Get_score(preds, y_test))
        Acc_Scores.append(Ensemble1.score(X_test, y_test))
        Recall_Scores.append(recall_score(y_test, preds))
        Precision_Scores.append(precision_score(y_test, preds))
        F1_Scores.append(f1_score(y_test, preds))
        print("")
        Fold = Fold +1

    print("Average Spearmans Score: ", np.mean(SP_Scores))
    print("Average Accuracy Score: ", np.mean(Acc_Scores))
    if np.mean(Acc_Scores) > Best['Acc']:
        Best['Acc'] = np.mean(Acc_Scores)
        Best['Model_Acc'] = 'Ensemble1'
        Best['Data_Acc'] = data

    print("Average Recall Score: ", np.mean(Recall_Scores))
    if np.mean(Recall_Scores) > Best['Recall']:
        Best['Recall'] = np.mean(Recall_Scores)
        Best['Model_Recall'] = 'Ensemble1'
        Best['Data_Recall'] = data

    print("Average Precision Score: ", np.mean(Precision_Scores))
    if np.mean(Precision_Scores) > Best['Precision']:
        Best['Precision'] = np.mean(Precision_Scores)
        Best['Model_Precision'] = 'Ensemble1'
        Best['Data_Precision'] = data

    print("Average F1 Score: ", np.mean(F1_Scores))
    if np.mean(F1_Scores) > Best['F1']:
        Best['F1'] = np.mean(F1_Scores)
        Best['Model_F1'] = 'Ensemble1'
        Best['Data_F1'] = data
    print("")

print("_____Best Scores and models_____")
for key in Best:
    print(key, ':', Best[key])

    

_____  ResNet _____
Fold:  1

Fold:  2

Fold:  3

Fold:  4

Fold:  5

Fold:  6

Average Spearmans Score:  0.3657923999191417
Average Accuracy Score:  0.6833333333333332
Average Recall Score:  0.6747011927303678
Average Precision Score:  0.6684856039609404
Average F1 Score:  0.6714154495165264

_____Best Scores and models_____
Model_Acc : SVC
Data_Acc : ResNet
Acc : 0.6836666666666665
Model_Recall : SVC
Data_Recall : ResNet
Recall : 0.6823068250947074
Model_Precision : SVC
Data_Precision : ResNet
Precision : 0.6706339780080982
Model_F1 : SVC
Data_F1 : ResNet
F1 : 0.6741068565402394


In [ ]:
model1 = BalancedBaggingClassifier(base_estimator=SVC(gamma='scale', C=1.0, kernel='rbf'))
model2 = RandomForestClassifier(class_weight='balanced', random_state=1)
model3 = GradientBoostingClassifier(random_state=1)
model4 = ExtraTreesClassifier(class_weight='balanced', random_state=1)
model5 = SVC(gamma='scale', C=1.0, kernel='rbf')   
    

for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)
    
    Fold = 1  
    SP_Scores = []
    Acc_Scores = []
    Recall_Scores = []
    Precision_Scores = []
    F1_Scores = []
    for train_index, test_index in kf.split(X):
        print("Fold: ", Fold)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        Ensemble2 = VotingClassifier(estimators=[('Bag', model1), ('RF', model2), ('GBC', model3), ('ETC', model4), ('SVC', model5)], voting='hard', weights=[1,1,2,1,3])
        Ensemble2 = Ensemble2.fit(X_train, y_train)
        preds = Ensemble2.predict(X_test)
        SP_Scores.append(Altered_Get_score(preds, y_test))
        Acc_Scores.append(Ensemble2.score(X_test, y_test))
        Recall_Scores.append(recall_score(y_test, preds))
        Precision_Scores.append(precision_score(y_test, preds))
        F1_Scores.append(f1_score(y_test, preds))
        print("")
        Fold = Fold +1

    print("Average Spearmans Score: ", np.mean(SP_Scores))
    print("Average Accuracy Score: ", np.mean(Acc_Scores))
    if np.mean(Acc_Scores) > Best['Acc']:
        Best['Acc'] = np.mean(Acc_Scores)
        Best['Model_Acc'] = 'Ensemble2'
        Best['Data_Acc'] = data

    print("Average Recall Score: ", np.mean(Recall_Scores))
    if np.mean(Recall_Scores) > Best['Recall']:
        Best['Recall'] = np.mean(Recall_Scores)
        Best['Model_Recall'] = 'Ensemble2'
        Best['Data_Recall'] = data

    print("Average Precision Score: ", np.mean(Precision_Scores))
    if np.mean(Precision_Scores) > Best['Precision']:
        Best['Precision'] = np.mean(Precision_Scores)
        Best['Model_Precision'] = 'Ensemble2'
        Best['Data_Precision'] = data

    print("Average F1 Score: ", np.mean(F1_Scores))
    if np.mean(F1_Scores) > Best['F1']:
        Best['F1'] = np.mean(F1_Scores)
        Best['Model_F1'] = 'Ensemble2'
        Best['Data_F1'] = data
    print("")

print("_____Best Scores and models_____")
for key in Best:
    print(key, ':', Best[key])

    

_____  ResNet _____
Fold:  1

Fold:  2

Fold:  3

Fold:  4

Fold:  5

Fold:  6

Average Spearmans Score:  0.34530966187476114
Average Accuracy Score:  0.6735000000000001
Average Recall Score:  0.6357697187005721
Average Precision Score:  0.6681557033870954
Average F1 Score:  0.6513038362142805

_____Best Scores and models_____
Model_Acc : SVC
Data_Acc : ResNet
Acc : 0.6836666666666665
Model_Recall : SVC
Data_Recall : ResNet
Recall : 0.6823068250947074
Model_Precision : SVC
Data_Precision : ResNet
Precision : 0.6706339780080982
Model_F1 : SVC
Data_F1 : ResNet
F1 : 0.6741068565402394


In [ ]:
model1 = BalancedBaggingClassifier(base_estimator=SVC(gamma='scale', C=1.0, kernel='rbf'))
model2 = RandomForestClassifier(class_weight='balanced', random_state=1)
model3 = GradientBoostingClassifier(random_state=1)
model4 = ExtraTreesClassifier(class_weight='balanced', random_state=1)
model5 = SVC(gamma='scale', C=1.0, kernel='rbf')   
    

for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)
    
    Fold = 1  
    SP_Scores = []
    Acc_Scores = []
    Recall_Scores = []
    Precision_Scores = []
    F1_Scores = []
    for train_index, test_index in kf.split(X):
        print("Fold: ", Fold)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        Ensemble3 = VotingClassifier(estimators=[('Bag', model1), ('RF', model2), ('GBC', model3), ('ETC', model4), ('SVC', model5)], voting='hard', weights=[1,0,2,0,3])
        Ensemble3 = Ensemble3.fit(X_train, y_train)
        preds = Ensemble3.predict(X_test)
        SP_Scores.append(Altered_Get_score(preds, y_test))
        Acc_Scores.append(Ensemble3.score(X_test, y_test))
        Recall_Scores.append(recall_score(y_test, preds))
        Precision_Scores.append(precision_score(y_test, preds))
        F1_Scores.append(f1_score(y_test, preds))

        print("")
        Fold = Fold +1

    print("Average Spearmans Score: ", np.mean(SP_Scores))
    print("Average Accuracy Score: ", np.mean(Acc_Scores))
    if np.mean(Acc_Scores) > Best['Acc']:
        Best['Acc'] = np.mean(Acc_Scores)
        Best['Model_Acc'] = 'Ensemble3'
        Best['Data_Acc'] = data

    print("Average Recall Score: ", np.mean(Recall_Scores))
    if np.mean(Recall_Scores) > Best['Recall']:
        Best['Recall'] = np.mean(Recall_Scores)
        Best['Model_Recall'] = 'Ensemble3'
        Best['Data_Recall'] = data

    print("Average Precision Score: ", np.mean(Precision_Scores))
    if np.mean(Precision_Scores) > Best['Precision']:
        Best['Precision'] = np.mean(Precision_Scores)
        Best['Model_Precision'] = 'Ensemble3'
        Best['Data_Precision'] = data

    print("Average F1 Score: ", np.mean(F1_Scores))
    if np.mean(F1_Scores) > Best['F1']:
        Best['F1'] = np.mean(F1_Scores)
        Best['Model_F1'] = 'Ensemble3'
        Best['Data_F1'] = data
    print("")

print("_____Best Scores and models_____")
for key in Best:
    print(key, ':', Best[key])

    

_____  ResNet _____
Fold:  1

Fold:  2

Fold:  3

Fold:  4

Fold:  5

Fold:  6

Average Spearmans Score:  0.3657923999191417
Average Accuracy Score:  0.6833333333333332
Average Recall Score:  0.6747011927303678
Average Precision Score:  0.6684856039609404
Average F1 Score:  0.6714154495165264

_____Best Scores and models_____
Model_Acc : SVC
Data_Acc : ResNet
Acc : 0.6836666666666665
Model_Recall : SVC
Data_Recall : ResNet
Recall : 0.6823068250947074
Model_Precision : SVC
Data_Precision : ResNet
Precision : 0.6706339780080982
Model_F1 : SVC
Data_F1 : ResNet
F1 : 0.6741068565402394


## Selected Classification Model
In the end a Support Vector Classifier with finetuned parameters was chosen as the best choice for the classification task achieving an average accuracy of 68% 

In [ ]:
Classification_Model = SVC(gamma='scale', C=1.0, kernel='rbf')   


X_train = Data['ResNet']['X']
Y_ST_train = Data['ResNet']['Y_ST']

Classification_Model = Classification_Model.fit(X_train, Y_ST_train)

# Regression model selection
In order to select a suitable regression model labels will have to be manually set as either "Correct" or "Wrong". If our classification model was trained on the train set and then used to predict values on the same train set this would lead to overfitting and we would achieve much higher spearman scores than what will be achieved on the test set

To do this 68% of the labels will be correctly assigned in our train set and our regression will be preformed using these features.

# Apply classification

In [ ]:
count = 0

## 68% of 6000 is 4080, 4080 datapoints will have the correct classification applied and the rest will be given an inocorrect classification.
def apply_Classification(x):
    global count
    if count <= 4080:
        if x == 1:
            count = count +1
            return np.ones(1)
        elif x ==0:
            count = count +1
            return np.zeros(1)
    elif count > 4080:
        if x == 1:
            count = count +1
            return np.zeros(1)
        elif x ==0:
            count = count +1
            return np.ones(1)

Dataset['ST_Classification'] = Dataset['ST_Round'].apply(lambda x: apply_Classification(x))


We can see in the dataframe below that the first few rows are correctly classified (ST_Round == ST_Classification) and the last few rows are incorrectly classified.

In [ ]:
Dataset

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,ST_Round,LT_Round,C3D,aesthetic_mean,aesthetic_median,ResNet,ST_Classification
0,video3,0.924,34,0.846,13,1,1,"[0.02024942, 0.0015778, 0.00082625, 0.00094509...","[0.18389, 0.713733, 0.202302, 0.610132, 0.1242...","[0.180874, 0.715626, 0.19893, 0.610453, 0.1216...","[0.27144504, 0.45918608, 0.035905946, 4.542650...",[1.0]
1,video4,0.923,33,0.667,12,1,0,"[0.000118, 0.00089075, 0.00018769, 4.543e-05, ...","[0.147255, 0.550718, 0.460299, 0.424981, 0.341...","[0.146076, 0.55007, 0.460729, 0.424397, 0.3418...","[0.9976212, 0.3619391, 0.2518191, 0.114101954,...",[1.0]
2,video6,0.863,33,0.700,10,0,0,"[0.01176522, 0.00074577, 0.00078353, 1.328e-05...","[0.063933, 0.826933, 0.214129, 0.719341, 0.130...","[0.063898, 0.827439, 0.211925, 0.720297, 0.129...","[0.7898796, 0.09988401, 0.020203931, 4.1379895...",[0.0]
3,video8,0.922,33,0.818,11,1,1,"[0.00022343, 0.00016499, 7.35e-06, 1.615e-05, ...","[0.367527, 0.133869, 0.746074, 0.169317, 0.696...","[0.380539, 0.134683, 0.746891, 0.171604, 0.697...","[0.0059119402, 0.27636862, 0.095939755, 0.1347...",[1.0]
4,video10,0.950,34,0.900,10,1,1,"[9.006e-05, 0.00061494, 0.00343634, 0.00128092...","[0.402878, 0.389358, 0.564687, 0.400378, 0.471...","[0.427258, 0.381448, 0.582746, 0.409475, 0.489...","[0.9489327, 1.3563356, 0.0, 0.33917212, 0.1328...",[1.0]
...,...,...,...,...,...,...,...,...,...,...,...,...
5995,video7488,0.921,33,1.000,9,1,1,"[0.35724622, 0.07598178, 0.00302532, 2.772e-05...","[0.4086, 0.194645, 0.713681, 0.283166, 0.65350...","[0.413464, 0.195451, 0.71372, 0.282848, 0.6534...","[0.5562924, 0.60748005, 0.0, 2.5067296, 0.0560...",[0.0]
5996,video7489,0.909,53,0.839,31,1,1,"[0.19280288, 0.00093812, 0.00506054, 1.35e-06,...","[0.265468, 0.188006, 0.451723, 0.160899, 0.415...","[0.264377, 0.192016, 0.448979, 0.164919, 0.412...","[0.5345125, 0.45983094, 0.012904617, 0.3444094...",[0.0]
5997,video7491,0.713,33,0.818,11,0,1,"[2.589e-05, 0.00012192, 2.781e-05, 2.608e-05, ...","[0.178087, 0.220956, 0.671676, 0.1793, 0.60967...","[0.177078, 0.221348, 0.67387, 0.180848, 0.6131...","[0.8621345, 0.29973385, 0.039232306, 3.5872233...",[1.0]
5998,video7492,0.954,34,1.000,16,1,1,"[0.26509121, 0.09653918, 5.971e-05, 0.00022735...","[0.113677, 0.585402, 0.496425, 0.475082, 0.352...","[0.113838, 0.585274, 0.496719, 0.474999, 0.352...","[0.531669, 0.3867722, 0.0, 4.139823, 0.2704557...",[0.0]


The dataframe needs to be shuffled next before it can be preprocessed and ready for K-fold cross validation

In [ ]:
shuffled_Dataset = shuffle(Dataset)

# Same as before except this time the memorability scores are our target variables

Y2 = shuffled_Dataset[['short-term_memorability','long-term_memorability']].values # targets

X2 = shuffled_Dataset[['C3D', 'aesthetic_median', 'ST_Classification']].values

# K-Fold cross validation

X_Attr2, Y_ST2, Y_LT2 = K_fold_processing(X2, Y2, 3)


Data = {
    'C3D, Aesthetic Median and ST classification':{
        'X':X_Attr2,
        'Y_ST':Y_ST2,
        'Y_LT':Y_LT2
    }
}


## Sklearn Models

A range of different sklearn regression models are then tested in order to 
1) See how well they preform with the new ST_Classification feature

2) Compare their Spearman scores and see the imporvement made by first classifying the ST memorability

### Original spearman scores

In [ ]:
Y3 = Dataset[['short-term_memorability','long-term_memorability']].values # targets

X3 = Dataset[['C3D', 'aesthetic_median']].values

# K-Fold cross validation

X_Attr3, Y_ST3, Y_LT3 = K_fold_processing(X3, Y3, 2)


Data2 = {
    'C3D and Aesthitic Median':{
        'X':X_Attr3,
        'Y_ST':Y_ST3,
        'Y_LT':Y_LT3
    }
}

models2 ={
    'RFR': RandomForestRegressor(),
    'BR': linear_model.BayesianRidge(alpha_1=1e-07, alpha_2=0.1, lambda_1=0.1, lambda_2= 0.1),
    'GBR': GradientBoostingRegressor(),
    'Bag': BaggingRegressor(),
    'ET': ExtraTreesRegressor(),
    'SVR': SVR(gamma='scale')   
}

print("_____ Short-Term Memorability _____")
for data in Data2:
    print('_____ ', data, '_____')
    X = Data2[data]['X']
    Y = Data2[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    for model in models2:
        print("_______________ Model: ", model, "_______________")

        temp = models2[model]
            
        Fold = 1  
        SP_Scores = []
        for train_index, test_index in kf.split(X):
            print("Fold: ", Fold)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            temp = temp.fit(X_train, y_train)
            preds = temp.predict(X_test)
            
            SP_Scores.append(Altered_Get_score(preds, y_test))
            Fold = Fold +1
        print("Average Spearmans Score: ", np.mean(SP_Scores))
        print("")

models3 ={
    'RFR': RandomForestRegressor(),
    'BR': linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001),
    'GBR': GradientBoostingRegressor(),
    'Bag': BaggingRegressor(),
    'ET': ExtraTreesRegressor(),
    'SVR': SVR(gamma='scale')   
}        
print("_____ Long-Term Memorability _____")
for data in Data2:
    print('_____ ', data, '_____')
    X = Data2[data]['X']
    Y = Data2[data]['Y_LT']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    for model in models3:
        print("_______________ Model: ", model, "_______________")

        temp = models3[model]
            
        Fold = 1  
        SP_Scores = []
        for train_index, test_index in kf.split(X):
            print("Fold: ", Fold)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            temp = temp.fit(X_train, y_train)
            preds = temp.predict(X_test)
            
            SP_Scores.append(Altered_Get_score(preds, y_test))
            Fold = Fold +1
        print("Average Spearmans Score: ", np.mean(SP_Scores))
        print("")


_____ Short-Term Memorability _____
_____  C3D and Aesthitic Median _____
_______________ Model:  RFR _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.3236878771195647

_______________ Model:  BR _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.27946503911028925

_______________ Model:  GBR _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.32735468683991026

_______________ Model:  Bag _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.24628366188373108

_______________ Model:  ET _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.3380253428662463

_______________ Model:  SVR _______________
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Average Spearmans Score:  0.24099424397201952

_____ Long-Term Memorability _____
_____  C3D and Aesthitic 

### New and improved spearman scores

As can be seen below there is a significant increase in the spearman score for the majority of models tested

In [ ]:
models ={
    'RFR': RandomForestRegressor(),
    'BR': linear_model.BayesianRidge(alpha_1=1e-07, alpha_2=0.1, lambda_1=0.1, lambda_2= 0.1),
    'GBR': GradientBoostingRegressor(),
    'Bag': BaggingRegressor(),
    'ET': ExtraTreesRegressor(),
    'SVR': SVR(gamma='scale')   
}

print("_____ Short-Term Memorability _____")
for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    for model in models:
        print("_______________ Model: ", model, "_______________")

        temp = models[model]
            
        Fold = 1  
        SP_Scores = []
        for train_index, test_index in kf.split(X):
            print("Fold: ", Fold)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            temp = temp.fit(X_train, y_train)
            preds = temp.predict(X_test)
            
            SP_Scores.append(Altered_Get_score(preds, y_test))
            Fold = Fold +1
        print("Average Spearmans Score: ", np.mean(SP_Scores))
        print("")

models2 ={
    'RFR': RandomForestRegressor(),
    'BR': linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001),
    'GBR': GradientBoostingRegressor(),
    'Bag': BaggingRegressor(),
    'ET': ExtraTreesRegressor(),
    'SVR': SVR(gamma='scale')   
}        
print("_____ Long-Term Memorability _____")
for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_LT']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    for model in models2:
        print("_______________ Model: ", model, "_______________")

        temp = models2[model]
            
        Fold = 1  
        SP_Scores = []
        for train_index, test_index in kf.split(X):
            print("Fold: ", Fold)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            temp = temp.fit(X_train, y_train)
            preds = temp.predict(X_test)
            
            SP_Scores.append(Altered_Get_score(preds, y_test))
            Fold = Fold +1
        print("Average Spearmans Score: ", np.mean(SP_Scores))
        print("")


_____ Short-Term Memorability _____
_____  C3D, Aesthetic Median and ST classification _____
_______________ Model:  RFR _______________
Fold:  1
The Spearman's correlation coefficient is: 0.445
Fold:  2
The Spearman's correlation coefficient is: 0.442
Fold:  3
The Spearman's correlation coefficient is: 0.403
Fold:  4
The Spearman's correlation coefficient is: 0.423
Fold:  5
The Spearman's correlation coefficient is: 0.456
Fold:  6
The Spearman's correlation coefficient is: 0.402
Average Spearmans Score:  0.42853997592899984

_______________ Model:  BR _______________
Fold:  1
The Spearman's correlation coefficient is: 0.431
Fold:  2
The Spearman's correlation coefficient is: 0.452
Fold:  3
The Spearman's correlation coefficient is: 0.392
Fold:  4
The Spearman's correlation coefficient is: 0.415
Fold:  5
The Spearman's correlation coefficient is: 0.441
Fold:  6
The Spearman's correlation coefficient is: 0.410
Average Spearmans Score:  0.42353473640138795

_______________ Model:  GBR __

## Ensemble approach
Several different combinations of models were tested and these were among the best preforming

In [ ]:
# Short term
estimators = [('RF', RandomForestRegressor(random_state=1)), ('ET', ExtraTreesRegressor(random_state=1)), ('GB', GradientBoostingRegressor()), ('BayR', linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001))]


print("_____ Short-Term Memorability _____")
for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    Scores=[]
    Fold = 1
    for train_index, test_index in kf.split(X):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        print("Fold: ", Fold, " of 6")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model = StackingRegressor(
                 estimators=estimators,
                 final_estimator=SVR(gamma='scale', C=0.1, epsilon=0.001, kernel='linear')  
                )
        model = model.fit(X_train, y_train)
        preds = model.predict(X_test)

        Scores.append(Altered_Get_score(preds, y_test))
        Fold = Fold+1

print("Average Spearmans Score: ", np.mean(Scores))

_____ Short-Term Memorability _____
_____  C3D, Aesthetic Median and ST classification _____
Fold:  1  of 6
The Spearman's correlation coefficient is: 0.474
Fold:  2  of 6
The Spearman's correlation coefficient is: 0.468
Fold:  3  of 6
The Spearman's correlation coefficient is: 0.424
Fold:  4  of 6
The Spearman's correlation coefficient is: 0.440
Fold:  5  of 6
The Spearman's correlation coefficient is: 0.478
Fold:  6  of 6
The Spearman's correlation coefficient is: 0.423
Average Spearmans Score:  0.45110650916204564


In [ ]:
# Short term
estimators = [('RF', RandomForestRegressor(random_state=1)), ('ET', ExtraTreesRegressor(random_state=1)), ('Bag', BaggingRegressor()), ('BayR', linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001))]


print("_____ Short-Term Memorability _____")
for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_ST']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    Scores=[]
    Fold = 1
    for train_index, test_index in kf.split(X):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        print("Fold: ", Fold, " of 6")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model = StackingRegressor(
                 estimators=estimators,
                 final_estimator=GradientBoostingRegressor() 
                )
        model = model.fit(X_train, y_train)
        preds = model.predict(X_test)

        Scores.append(Altered_Get_score(preds, y_test))
        Fold = Fold+1

print("Average Spearmans Score: ", np.mean(Scores))

_____ Short-Term Memorability _____
_____  C3D, Aesthetic Median and ST classification _____
Fold:  1  of 6
The Spearman's correlation coefficient is: 0.462
Fold:  2  of 6
The Spearman's correlation coefficient is: 0.458
Fold:  3  of 6
The Spearman's correlation coefficient is: 0.415
Fold:  4  of 6
The Spearman's correlation coefficient is: 0.417
Fold:  5  of 6
The Spearman's correlation coefficient is: 0.463
Fold:  6  of 6
The Spearman's correlation coefficient is: 0.418
Average Spearmans Score:  0.43915289321811785


In [ ]:
# Long term
estimators = [('RF', RandomForestRegressor(random_state=1)), ('ET', ExtraTreesRegressor(random_state=1)), ('GB', GradientBoostingRegressor()), ('BayR', linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001))]

model = StackingRegressor(
     estimators=estimators,
     final_estimator=SVR(gamma='scale', C=0.1, epsilon=0.001, kernel='linear')  
    )


print("_____ Long-Term Memorability _____")
for data in Data:
    print('_____ ', data, '_____')
    X = Data[data]['X']
    Y = Data[data]['Y_LT']
    kf = KFold(n_splits=6)
    kf.get_n_splits(X, Y)

    Scores=[]
    Fold = 1
    for train_index, test_index in kf.split(X):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        print("Fold: ", Fold, " of 6")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model = model.fit(X_train, y_train)
        preds = model.predict(X_test)

        Scores.append(Altered_Get_score(preds, y_test))
        Fold = Fold+1

print("Average Spearmans Score: ", np.mean(Scores))

_____ Short-Term Memorability _____
_____  C3D, Aesthetic Median and ST classification _____
Fold:  1  of 6
The Spearman's correlation coefficient is: 0.182
Fold:  2  of 6
The Spearman's correlation coefficient is: 0.181
Fold:  3  of 6
The Spearman's correlation coefficient is: 0.103
Fold:  4  of 6
The Spearman's correlation coefficient is: 0.164
Fold:  5  of 6
The Spearman's correlation coefficient is: 0.222
Fold:  6  of 6
The Spearman's correlation coefficient is: 0.156
Average Spearmans Score:  0.1679405195154903


## Chosen Regression Models

### Short-term

In [ ]:
from sklearn.ensemble import StackingRegressor

X = X_Attr2
Y = Y_ST2

# Short term
estimators = [('RF', RandomForestRegressor(random_state=1)), ('ET', ExtraTreesRegressor(random_state=1)), ('GB', GradientBoostingRegressor()), ('BayR', linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001))]

ST_Regression_model = StackingRegressor(
         estimators=estimators,
         final_estimator=SVR(gamma='scale', C=0.1, epsilon=0.001, kernel='linear')  
        )
ST_Regression_model = ST_Regression_model.fit(X, Y)



NameError: ignored

### Long-term

In [ ]:
from sklearn.ensemble import StackingRegressor

X = X_Attr2
Y = Y_LT2

# Short term
estimators = [('RF', RandomForestRegressor(random_state=1)), ('ET', ExtraTreesRegressor(random_state=1)), ('GB', GradientBoostingRegressor()), ('BayR', linear_model.BayesianRidge(alpha_1=0.1, alpha_2=1e-07, lambda_1=1e-07, lambda_2= 0.001))]

LT_Regression_model = StackingRegressor(
         estimators=estimators,
         final_estimator=SVR(gamma='scale', C=0.1, epsilon=0.001, kernel='linear')  
        )
LT_Regression_model = LT_Regression_model.fit(X, Y)



# Test-set Predictions

Now the same process will be followed in order to generate a set of predictioons for the 2000 videos in the test set

In [ ]:
base_dir = "C:/Users/rugby/Datasets/CA684"
train_dir = os.path.join(base_dir, "Dev-set/Train-Set")
test_dir = os.path.join(base_dir, "Test-Set")
os.chdir(test_dir)
os.getcwd()

'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set'

Finding where the features are located relevant to our current directory and adding the path to each to a dictionary object where the keys are the feature name for easy access

In [ ]:
test_feature_folders = {}
for test_feature_folder in os.listdir(os.getcwd()):
    test_feature_folders[str(test_feature_folder)] = os.path.join(os.getcwd(),str(test_feature_folder))
  

In [ ]:
test_feature_folders

{'aesthetic_vis_features_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\aesthetic_vis_features_test',
 'C3D_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\C3D_test',
 'Captions_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\Captions_test',
 'ColorHistogram_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\ColorHistogram_test',
 'Ground-truth_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\Ground-truth_test',
 'HMP_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\HMP_test',
 'HOG_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\HOG_test',
 'Inception_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\Inception_test',
 'LBP_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\LBP_test',
 'ORB_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\ORB_test',
 'README.txt': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\README.txt',
 'ResNet_test': 'C:\\Users\\rugby\\Datasets\\CA684\\Test-Set\\ResNet_test'}

# **Loading Test-set CSV:**
Loading in the ground truths first as they are among the most important

In [ ]:
groundTruthsTest_file = os.path.join(test_dir, "Ground-truth_test/ground_truth_template.csv")
groundTruthsTestdf = pd.read_csv(groundTruthsTest_file)
groundTruthsTestdf['video'] = groundTruthsTestdf['video'].str.rstrip('.webm')
groundTruthsTestdf

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video7494,NaN,33,NaN,12
1,video7495,NaN,34,NaN,10
2,video7496,NaN,32,NaN,13
3,video7497,NaN,33,NaN,10
4,video7498,NaN,33,NaN,10
...,...,...,...,...,...
1995,video10004,NaN,34,NaN,17
1996,video10005,NaN,34,NaN,9
1997,video10006,NaN,34,NaN,12
1998,video10007,NaN,34,NaN,12


## Test-set Feature dictionaries
Next we will go through our features and load each of them into their own dictionary object.

Similar to how it was done for the Dev-set

In [ ]:
os.chdir(test_feature_folders['C3D_test'])

test_C3D_Dict = {"video":[],"C3D":[]}

for file in sorted(os.listdir(os.getcwd())):
    test_C3D_Dict["video"].append(os.path.splitext(file)[0])
    test_C3D_Dict["C3D"].append(np.array(read_C3D(file)))

In [ ]:
test_aesthetic_mean = os.path.join(test_feature_folders['aesthetic_vis_features_test'], 'mean', )
test_aesthetic_median = os.path.join(test_feature_folders['aesthetic_vis_features_test'], 'median', )

os.chdir(test_aesthetic_mean)

test_aesthetic_mean_Dict = {"video":[],"aesthetic_mean":[]}

for file in os.listdir(os.getcwd()):
    test_aesthetic_mean_Dict["video"].append(os.path.splitext(file)[0])
    test_aesthetic_mean_Dict["aesthetic_mean"].append(np.array(read_aesthetic(file)))
    
os.chdir(test_aesthetic_median)

test_aesthetic_median_Dict = {"video":[],"aesthetic_median":[]}

for file in os.listdir(os.getcwd()):
    test_aesthetic_median_Dict["video"].append(os.path.splitext(file)[0])
    test_aesthetic_median_Dict["aesthetic_median"].append(np.array(read_aesthetic(file)))

In [ ]:
os.chdir(test_feature_folders['ResNet_test'])

test_ResNet_Dict = {"video":[],"ResNet":[]}

for file in sorted(os.listdir(os.getcwd())):
    test_ResNet_Dict["video"].append(os.path.splitext(file)[0])
    test_ResNet_Dict["ResNet"].append(read_RESNET(file))

In [ ]:
test_datasets = [test_C3D_Dict, test_aesthetic_mean_Dict, test_aesthetic_median_Dict, test_ResNet_Dict]

In [ ]:
Test_Dataset = groundTruthsTestdf

for data in test_datasets:
    temp = pd.DataFrame(data)
    temp
    Test_Dataset = Test_Dataset.merge(temp, left_on='video', right_on='video')

In [ ]:
Test_Dataset

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,C3D,aesthetic_mean,aesthetic_median,ResNet
0,video7494,NaN,33,NaN,12,"[0.00013576, 0.0002133, 0.00124759, 1.149e-05,...","[0.15188, 0.435622, 0.385821, 0.324642, 0.3163...","[0.151128, 0.428798, 0.388939, 0.319523, 0.319...","[0.0, 0.6642892, 0.40391892, 0.08664888, 0.266..."
1,video7495,NaN,34,NaN,10,"[0.00012659, 2.377e-05, 0.04950115, 2.418e-05,...","[0.346129, 0.39686, 0.518731, 0.417674, 0.4222...","[0.351501, 0.38728, 0.52406, 0.410809, 0.42906...","[0.3602615, 0.205313, 0.6395493, 0.42662442, 0..."
2,video7496,NaN,32,NaN,13,"[0.48898152, 0.00058536, 2.454e-05, 0.00016031...","[0.126344, 0.225354, 0.479295, 0.172265, 0.432...","[0.126592, 0.225055, 0.479929, 0.172715, 0.432...","[0.3960949, 0.28923368, 0.07051848, 2.962809, ..."
3,video7497,NaN,33,NaN,10,"[0.24925078, 0.00101763, 8.02e-06, 1.92e-05, 2...","[0.217836, 0.338128, 0.633791, 0.245464, 0.529...","[0.218887, 0.337801, 0.633643, 0.245733, 0.529...","[0.41966626, 0.20611148, 0.0, 3.8670971, 0.244..."
4,video7498,NaN,33,NaN,10,"[0.00989415, 3.413e-05, 0.00261261, 2.1e-06, 1...","[0.436468, 0.292819, 0.4736, 0.270421, 0.40387...","[0.438498, 0.290777, 0.474659, 0.267142, 0.403...","[1.0317943, 0.595292, 0.5026228, 1.083852, 0.0..."
...,...,...,...,...,...,...,...,...,...
1995,video10004,NaN,34,NaN,17,"[0.05058656, 0.00623968, 0.11702564, 0.0005364...","[0.401823, 0.334808, 0.436243, 0.329913, 0.352...","[0.402608, 0.337565, 0.435286, 0.33409, 0.3524...","[2.9015408, 1.0742561, 0.699099, 0.039030563, ..."
1996,video10005,NaN,34,NaN,9,"[0.00038171, 6.61e-05, 2.447e-05, 3.05e-06, 1....","[0.174526, 0.375534, 0.404438, 0.327168, 0.345...","[0.173931, 0.375612, 0.402786, 0.329354, 0.342...","[1.414171, 0.11000307, 0.041044723, 2.8321652,..."
1997,video10006,NaN,34,NaN,12,"[0.59629649, 0.03444825, 0.0009805, 6e-08, 1e-...","[0.299844, 0.207785, 0.502194, 0.140744, 0.456...","[0.298056, 0.211533, 0.500626, 0.143247, 0.454...","[0.26084593, 0.12867452, 0.055413503, 1.148036..."
1998,video10007,NaN,34,NaN,12,"[0.00025415, 0.00021214, 0.02341845, 0.0104768...","[0.293313, 0.207653, 0.687289, 0.216966, 0.619...","[0.293256, 0.207618, 0.687723, 0.216891, 0.619...","[2.0340781, 0.0, 0.11969166, 0.21145138, 0.029..."


Follow the same procedure as when the model was originally trained, this involves preforming PCA on the ResNet features so that they match the dimensions that the model is expecting

In [ ]:
# Here we must apply PCA to the ResNet feature just like we did when we were training the classifier in the first place.


def apply_PCA(x):
    x = x.reshape(1, -1)
#     print(x.shape)
    temp = pca.transform(x)
    return temp

Test_Dataset['ResNetPCA'] = Test_Dataset['ResNet'].apply(lambda x: apply_PCA(x))

Test_Dataset

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,C3D,aesthetic_mean,aesthetic_median,ResNet,ResNetPCA
0,video7494,NaN,33,NaN,12,"[0.00013576, 0.0002133, 0.00124759, 1.149e-05,...","[0.15188, 0.435622, 0.385821, 0.324642, 0.3163...","[0.151128, 0.428798, 0.388939, 0.319523, 0.319...","[0.0, 0.6642892, 0.40391892, 0.08664888, 0.266...","[[27.518766, 0.08659488, -3.195951, 0.9473192,..."
1,video7495,NaN,34,NaN,10,"[0.00012659, 2.377e-05, 0.04950115, 2.418e-05,...","[0.346129, 0.39686, 0.518731, 0.417674, 0.4222...","[0.351501, 0.38728, 0.52406, 0.410809, 0.42906...","[0.3602615, 0.205313, 0.6395493, 0.42662442, 0...","[[25.280325, -5.0829916, 19.465738, -3.2440116..."
2,video7496,NaN,32,NaN,13,"[0.48898152, 0.00058536, 2.454e-05, 0.00016031...","[0.126344, 0.225354, 0.479295, 0.172265, 0.432...","[0.126592, 0.225055, 0.479929, 0.172715, 0.432...","[0.3960949, 0.28923368, 0.07051848, 2.962809, ...","[[-28.642748, -25.548864, -12.108932, -11.1611..."
3,video7497,NaN,33,NaN,10,"[0.24925078, 0.00101763, 8.02e-06, 1.92e-05, 2...","[0.217836, 0.338128, 0.633791, 0.245464, 0.529...","[0.218887, 0.337801, 0.633643, 0.245733, 0.529...","[0.41966626, 0.20611148, 0.0, 3.8670971, 0.244...","[[-32.18834, -15.102641, -0.055840716, 6.95389..."
4,video7498,NaN,33,NaN,10,"[0.00989415, 3.413e-05, 0.00261261, 2.1e-06, 1...","[0.436468, 0.292819, 0.4736, 0.270421, 0.40387...","[0.438498, 0.290777, 0.474659, 0.267142, 0.403...","[1.0317943, 0.595292, 0.5026228, 1.083852, 0.0...","[[16.272593, 1.2831819, 32.20946, 4.2547565, -..."
...,...,...,...,...,...,...,...,...,...,...
1995,video10004,NaN,34,NaN,17,"[0.05058656, 0.00623968, 0.11702564, 0.0005364...","[0.401823, 0.334808, 0.436243, 0.329913, 0.352...","[0.402608, 0.337565, 0.435286, 0.33409, 0.3524...","[2.9015408, 1.0742561, 0.699099, 0.039030563, ...","[[12.347978, -9.903356, -4.5053425, 17.755169,..."
1996,video10005,NaN,34,NaN,9,"[0.00038171, 6.61e-05, 2.447e-05, 3.05e-06, 1....","[0.174526, 0.375534, 0.404438, 0.327168, 0.345...","[0.173931, 0.375612, 0.402786, 0.329354, 0.342...","[1.414171, 0.11000307, 0.041044723, 2.8321652,...","[[-30.751497, -1.1279987, -0.6496754, -0.42172..."
1997,video10006,NaN,34,NaN,12,"[0.59629649, 0.03444825, 0.0009805, 6e-08, 1e-...","[0.299844, 0.207785, 0.502194, 0.140744, 0.456...","[0.298056, 0.211533, 0.500626, 0.143247, 0.454...","[0.26084593, 0.12867452, 0.055413503, 1.148036...","[[-38.928253, -12.818127, 10.368459, 3.923568,..."
1998,video10007,NaN,34,NaN,12,"[0.00025415, 0.00021214, 0.02341845, 0.0104768...","[0.293313, 0.207653, 0.687289, 0.216966, 0.619...","[0.293256, 0.207618, 0.687723, 0.216891, 0.619...","[2.0340781, 0.0, 0.11969166, 0.21145138, 0.029...","[[5.317804, -9.489019, 37.526974, 44.837975, 2..."


# Apply classification

In [ ]:
def apply_Classification(x):
    return Classification_Model.predict(x)


Test_Dataset['ST_Classification'] = Test_Dataset['ResNetPCA'].apply(lambda x: apply_Classification(x))

Test_Dataset

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,C3D,aesthetic_mean,aesthetic_median,ResNet,ResNetPCA,ST_Classification
0,video7494,NaN,33,NaN,12,"[0.00013576, 0.0002133, 0.00124759, 1.149e-05,...","[0.15188, 0.435622, 0.385821, 0.324642, 0.3163...","[0.151128, 0.428798, 0.388939, 0.319523, 0.319...","[0.0, 0.6642892, 0.40391892, 0.08664888, 0.266...","[[27.518766, 0.08659488, -3.195951, 0.9473192,...",[1]
1,video7495,NaN,34,NaN,10,"[0.00012659, 2.377e-05, 0.04950115, 2.418e-05,...","[0.346129, 0.39686, 0.518731, 0.417674, 0.4222...","[0.351501, 0.38728, 0.52406, 0.410809, 0.42906...","[0.3602615, 0.205313, 0.6395493, 0.42662442, 0...","[[25.280325, -5.0829916, 19.465738, -3.2440116...",[0]
2,video7496,NaN,32,NaN,13,"[0.48898152, 0.00058536, 2.454e-05, 0.00016031...","[0.126344, 0.225354, 0.479295, 0.172265, 0.432...","[0.126592, 0.225055, 0.479929, 0.172715, 0.432...","[0.3960949, 0.28923368, 0.07051848, 2.962809, ...","[[-28.642748, -25.548864, -12.108932, -11.1611...",[1]
3,video7497,NaN,33,NaN,10,"[0.24925078, 0.00101763, 8.02e-06, 1.92e-05, 2...","[0.217836, 0.338128, 0.633791, 0.245464, 0.529...","[0.218887, 0.337801, 0.633643, 0.245733, 0.529...","[0.41966626, 0.20611148, 0.0, 3.8670971, 0.244...","[[-32.18834, -15.102641, -0.055840716, 6.95389...",[1]
4,video7498,NaN,33,NaN,10,"[0.00989415, 3.413e-05, 0.00261261, 2.1e-06, 1...","[0.436468, 0.292819, 0.4736, 0.270421, 0.40387...","[0.438498, 0.290777, 0.474659, 0.267142, 0.403...","[1.0317943, 0.595292, 0.5026228, 1.083852, 0.0...","[[16.272593, 1.2831819, 32.20946, 4.2547565, -...",[0]
...,...,...,...,...,...,...,...,...,...,...,...
1995,video10004,NaN,34,NaN,17,"[0.05058656, 0.00623968, 0.11702564, 0.0005364...","[0.401823, 0.334808, 0.436243, 0.329913, 0.352...","[0.402608, 0.337565, 0.435286, 0.33409, 0.3524...","[2.9015408, 1.0742561, 0.699099, 0.039030563, ...","[[12.347978, -9.903356, -4.5053425, 17.755169,...",[0]
1996,video10005,NaN,34,NaN,9,"[0.00038171, 6.61e-05, 2.447e-05, 3.05e-06, 1....","[0.174526, 0.375534, 0.404438, 0.327168, 0.345...","[0.173931, 0.375612, 0.402786, 0.329354, 0.342...","[1.414171, 0.11000307, 0.041044723, 2.8321652,...","[[-30.751497, -1.1279987, -0.6496754, -0.42172...",[1]
1997,video10006,NaN,34,NaN,12,"[0.59629649, 0.03444825, 0.0009805, 6e-08, 1e-...","[0.299844, 0.207785, 0.502194, 0.140744, 0.456...","[0.298056, 0.211533, 0.500626, 0.143247, 0.454...","[0.26084593, 0.12867452, 0.055413503, 1.148036...","[[-38.928253, -12.818127, 10.368459, 3.923568,...",[1]
1998,video10007,NaN,34,NaN,12,"[0.00025415, 0.00021214, 0.02341845, 0.0104768...","[0.293313, 0.207653, 0.687289, 0.216966, 0.619...","[0.293256, 0.207618, 0.687723, 0.216891, 0.619...","[2.0340781, 0.0, 0.11969166, 0.21145138, 0.029...","[[5.317804, -9.489019, 37.526974, 44.837975, 2...",[1]


In [ ]:
from keras.utils import to_categorical
from sklearn.utils import shuffle


# Same as before except this time the memorability scores are our target variables

Y_test = Test_Dataset[['short-term_memorability','long-term_memorability']].values # targets

X_test = Test_Dataset[['C3D', 'aesthetic_median', 'ST_Classification']].values

# K-Fold cross validation

X_Attr_test, Y_ST_test, Y_LT_test = K_fold_processing(X_test, Y_test, 3)


Data = {
    'C3D, Aesthetic Median and ST classification':{
        'X':X_Attr_test,
        'Y_ST':Y_ST_test,
        'Y_LT':Y_LT_test
    }
}


# Making Predictions

In [ ]:
ST_Preds = ST_Regression_model.predict(X_Attr_test)
LT_Preds = LT_Regression_model.predict(X_Attr_test)

In [ ]:
os.chdir('C:\\Users\\rugby\\OneDrive\\Documents\\DCU- Artificial Intelligence\\Machine Learning\\Assignment\\Code\\Pickle Files')
os.getcwd()

'C:\\Users\\rugby\\OneDrive\\Documents\\DCU- Artificial Intelligence\\Machine Learning\\Assignment\\Code\\Pickle Files'

Once made predictions are written to text files to be manually added to the ground truths csv file

In [ ]:
with open("ST_Preds.txt", "w") as txt_file:
    for line in ST_Preds:
        txt_file.write(str(line) + "\n")

In [ ]:
with open("LT_Preds.txt", "w") as txt_file:
    for line in LT_Preds:
        txt_file.write(str(line) + "\n")